In [ ]:
import os
import openai
import time

openai.organization = "org-8ee5MPlwE4fOKQYPUNGa0KO7"
openai.api_key = "sk-"

# 모델 - GPT 3.5 Turbo 지정
# => 모델 목록은 : https://platform.openai.com/docs/models/gpt-4 참조
model = "gpt-3.5-turbo"#"gpt-4"#"gpt-3.5-turbo" #gpt-4-0314

In [ ]:

# 처음 역활을 부여하는 쿼리 (1번만 호출되는 쿼리임)
def role_set_query():
   
    start_time = time.time()

    #ROLE_PROMPT = "쇼핑 호스트 역활을 부여합니다.질문에 대해 쇼핑 호스트 처럼 답변하고 요청 질문을 해주세요.그리고 키워드 단어들을 'key=단어,단어,...'형태로 출력해 주세요."
    #ROLE_PROMPT = "질문에 대한 답을 찾아서 한문장으로 요약해서 간단히 응답해주세요. 이때 빠르게 응답하도록 응답 내용은 30자를 넘지 말아야 합니다."

    # 메시지 설정
    MESSAGES = [
                {"role": "system", "content": "You are a helful assistant."},
                #{"role": "system", "content": "질문에 대해 요약해줘"},
                #{"role": "user", "content" : "How are you?"},
                #{"role": "assistant", "content" : "I am doing well"},
                {"role": "user", "content": ROLE_PROMPT}
            ]

    # ChatGPT-API 호출하기
    response = openai.ChatCompletion.create(
            model=model,
            messages=MESSAGES,
            max_tokens=500, # 토큰 수 
            temperature=2,  # temperature 0~2 범위 : 작을수록 정형화된 답변, 클수록 유연한 답변(2는 엉뚱한 답변을 하므로, 1.5정도가 좋은것 같음=기본값은=1)
            top_p=0.1, # 기본값은 1 (0.1이라고 하면 10% 토큰들에서 출력 토큰들을 선택한다는 의미)
            frequency_penalty=0.5, # 일반적으로 나오지 않는 단어를 억제하는 정도
            presence_penalty=0.5, # 동일한 단어나 구문이 반복되는 것을 억제하는 정도
        )

    print(response)
    print()
    answer = response['choices'][0]['message']['content']
    print(f'A:{answer}')
    print()    
    
    # 소요된 시간을 계산합니다.
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'응답시간:{elapsed_time}')
    
    return answer 

def generate_text_GPT():
    
    prompt = input("질문을 입력하세요")
    #print(f'\n\nQ:{prompt}\n\n')
    
    start_time = time.time()
    
    #print(f'len(messages):{len(messages)}') 
    #print()
    
    # 메시지 설정
    MESSAGES = [
            {"role": "system", "content": "답은 간략히 한문장으로 만들어 주세요."},
            {"role": "user", "content": prompt}
        ]

    # ChatGPT-API 호출하기
    response = openai.ChatCompletion.create(
        model=model,
        messages=MESSAGES,
        max_tokens=512, # 토큰 수 
        temperature=1,  # temperature 0~2 범위 : 작을수록 정형화된 답변, 클수록 유연한 답변(2는 엉뚱한 답변을 하므로, 1.5정도가 좋은것 같음=기본값은=1)
        top_p=0.1, # 기본값은 1 (0.1이라고 하면 10% 토큰들에서 출력 토큰들을 선택한다는 의미)
        frequency_penalty=0.5, # 일반적으로 나오지 않는 단어를 억제하는 정도
        presence_penalty=0.5, # 동일한 단어나 구문이 반복되는 것을 억제하는 정도
        #stop=["다.","다!"] # . 나오면 중단
    )

    print(response)
    print()
    
 
    # 소요된 시간을 계산합니다.
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'응답시간:{elapsed_time}')
    print()
    #answer = response['choices'][0]['message']['content']+'다.'  # '다.'로 끝나므로, 다를 붙여줌
    answer = response['choices'][0]['message']['content']
    print(answer)
    print()
    return answer

#--------------------------
def run_query_loop():
    
   # answer = role_set_query()
    
    while True:
        try:
            generate_text_GPT()
            #answer = handle_query(answer)
        except KeyboardInterrupt:
            return
#--------------------------  

In [ ]:
run_query_loop()